In [1]:
VERSION = '99_v1_02_08'

In [2]:
model_list = [    
    #{'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':0.5}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':0.5}, #deberta-base, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':1}, #longformer-large, seed100, split00_v2_07
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
    pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
    pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5926


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,Ineffective_1,Adequate_1,Effective_1,Ineffective_2,Adequate_2,Effective_2,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.004350,0.305959,0.023024,1,0.085691,0.010763,0.308277,0.014293,0.002334,0.270700,0.060300,0.017446,0.884937,0.097617
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.018665,0.305458,0.009210,1,0.087332,0.017491,0.307105,0.008737,0.023707,0.296091,0.013536,0.059863,0.908653,0.031483
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.046903,0.282717,0.003714,1,0.164698,0.084472,0.245645,0.003216,0.024279,0.296698,0.012356,0.155654,0.825060,0.019286
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.111466,0.218990,0.002878,1,0.420119,0.079564,0.249848,0.003921,0.085017,0.239872,0.008444,0.276048,0.708709,0.015243
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.082580,0.249347,0.001406,1,0.290297,0.114735,0.216563,0.002036,0.055173,0.268754,0.009406,0.252488,0.734664,0.012848
